<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/teste_correlacao_issue_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [2]:
print('Install dependencies...')
print('Update apt. ')
!sudo apt update > apt_update.log
print('Install software-properties-common.')
!sudo apt install software-properties-common > spc.log
print('Add ppa-git-core repository.')
!sudo add-apt-repository -y ppa:git-core/ppa > ppa_git_core.log
print('Install the latest version of git.')
!sudo apt install -y git > install_git.log
!git --version > git_version.log
print('Install pydriller')
!pip3 install pydriller > install_pydriller.log
!pip3 list | grep PyDriller > versao_pydriller.log
print('Install gitpython.')
!pip3 install gitpython > install_gitpython.log
print('Install Jira Python lib.')
!pip install jira > install_jira_python.log
print('All depenpencies installed!')
!cat *.log > install.log
print('Details in install.log')

Install dependencies...
Update apt. 


Install software-properties-common.


Add ppa-git-core repository.
Install the latest version of git.


Install pydriller
Install gitpython.
Install Jira Python lib.
All depenpencies installed!
cat: install.log: input file is output file
Details in install.log


In [3]:
# 1. import libraries and modules
import pandas as pd
import pydriller
from jira import JIRA
import os
import subprocess
import requests
import threading
import datetime
import configparser # para manipular arquivos .conf
import tqdm

# Apoio

In [4]:
# 2. Define variables
github_repo_url = 'https://github.com/apache/cassandra.git'
# jira_url = 'https://issues.apache.org/jira'
SERVIDOR_JIRA = 'https://issues.apache.org/jira'

In [5]:
'''
Create a file credentials.conf with the following content:
[credentials]
username = ?
password = ?
'''
config = configparser.ConfigParser()
config.read('credentials.conf')

username = config['credentials']['username']
password = config['credentials']['password']

try:
  # Create a JIRA client instance
  jira = JIRA(
      server=SERVIDOR_JIRA,  # Replace with the Cassandra JIRA server URL
      basic_auth=(username, password),  # Replace with your JIRA credentials
  )
  print(f'Servidor {SERVIDOR_JIRA} acessado com sucesso!')
except Exception as ex:
  print(f'Erro ao acessar o Jira: {str(ex)}')

Servidor https://issues.apache.org/jira acessado com sucesso!


In [6]:
from datetime import datetime, timedelta
import subprocess

#!cd cassandra && git rev-list --count HEAD

def get_total_commit_count(repository_path):
    command = ["git", "rev-list", "--count", "HEAD"]
    result = subprocess.run(command, cwd=repository_path, stdout=subprocess.PIPE, text=True)
    total_commit_count = int(result.stdout.strip())
    return total_commit_count

def generate_intervals_between_dates(date1, date2, distance=120):
  # Define the start and end dates
  start_date = datetime(date1[0], date1[1], date1[2])
  end_date = datetime(date2[0], date2[1], date2[2])

  # Define the interval in days
  interval_days = distance

  # Initialize a list to store the intervals
  intervals = []

  # Initialize the current date as the start date
  current_date = start_date

  # Loop to generate intervals until the current date is less than or equal to the end date
  while current_date < end_date:
      interval = (current_date, current_date + timedelta(days=interval_days - 1))
      intervals.append(interval)
      current_date += timedelta(days=interval_days)

  return intervals

def convert_interval_dates(dates):
  list_interval_dates = []
  for each in dates:
    date1 = each[0]
    # Convert the date to a string in the format "YYYY/MM/DD".
    str_date1 = date1.strftime("%Y/%m/%d")
    date2 = each[1]
    str_date2 = date2.strftime("%Y/%m/%d")
    elemento = str_date1, str_date2
    list_interval_dates.append(elemento)

  return list_interval_dates

# gera uma lista de strings
def gera_lista_sentencas(project_name, dates):
  lista_sentencas = []
  for each in dates:
    str_date1 = each[0].strftime("%Y/%m/%d")
    str_date2 = each[1].strftime("%Y/%m/%d")
    sentenca = f'project={project_name} and created>="{str_date1}" and created<="{str_date2}"'
    lista_sentencas.append(sentenca)

  return lista_sentencas

def lista_acesso_jira(lista_sentencas):
  print('Aguarde...')
  t1 = datetime.now()
  lista_issues_by_date = []
  for each in lista_sentencas:
    issues_by_date_temp = jira.search_issues(each,maxResults=1000)
    print(each, len(issues_by_date_temp))
    lista_issues_by_date.append(issues_by_date_temp)
  t2 = datetime.now()
  print(t2)
  print(f'Tempo da consulta: {t2-t1}')

  return lista_issues_by_date

# extrai o issue tracker salvo na mensagem de cada commit
def extract_cassandra_issue(text):
  """
  Extract the CASSANDRA-XYZ issue from the text.

  Args:
    text: The text to extract the issue from.

  Returns:
    The issue number, or None if the issue cannot be found.
  """

  issues_number = []

  lines = text.split('\n')
  for line in lines:
    if 'CASSANDRA-' in line :
      issue_number = line.split('-')[1]
      issue_number = 'CASSANDRA-' + issue_number
      issues_number.append(issue_number)
  return issues_number

def get_modified_files_from_commit(key):
  list_files = []
  MESSAGE = 0
  MODIFIED_FILES = 1
  QTD_MODIFIED_FILES = 2
  for each in key:
    if each in dict_commit_modified_files.keys():
      if dict_commit_modified_files[each] is not None:
        list_files.append(dict_commit_modified_files[each][MODIFIED_FILES])
  return list_files

# Extrai Commits do repositório

In [8]:
# Mostra todos os commits
!cd cassandra && git log --pretty="%H;%ai;%s" > all_commits_full.txt
!cd cassandra && git log --pretty="%H %s" > all_commits_msg.txt
!echo "10 primeiros commits all_commits_msg.txt"
!cd cassandra && head all_commits_msg.txt
!echo "..."

# Total de commits registrados entre as versões cassandra-3.0.0...cassandra-3.11.11
!echo "Total de commits registrados "
!cd cassandra && cat all_commits_msg.txt | wc -l

10 primeiros commits all_commits_msg.txt
830f88d4ab2aaa1f7b88b90747a808d0a4e2a708 Merge branch 'cassandra-5.0' into trunk
6708499e4c930f7733894e6076b925fb151d9251 Remove metrics-reporter-config dependency
1959502d8b16212479eecb076c89945c3f0f180c Merge branch 'cassandra-5.0' into trunk
00c0cbb958db570e759e685da85dc840de7dd739 Merge branch 'cassandra-4.1' into cassandra-5.0
3d220b5a6e2528b7b9f89074fe48dc6556977dc1 Merge branch 'cassandra-4.0' into cassandra-4.1
b2093ef3216bb38aa56fd7d9902412a6f49232bc Select automatic native port when starting the server in CQLTester
759f2b29324aa8e55be736583961845a329b8cbd Merge branch 'cassandra-5.0' into trunk
30d540d797ef738ff47e7257887decc4c7420c19 Merge branch 'cassandra-4.1' into cassandra-5.0
b0530276809f2cec3b170446396b4bd0869948e7 Set the delay to 0 for unit tests and fix GuardrailDiskUsageTest
a67f4904f7cfda90239d285333903c42ec1596b2 Merge branch 'cassandra-5.0' into trunk
...
Total de commits registrados 
29152


In [26]:
!head /content/cassandra/all_commits_full.txt -n 1
!tail /content/cassandra/all_commits_full.txt -n 1

830f88d4ab2aaa1f7b88b90747a808d0a4e2a708;2023-09-12 10:22:23 -0400;Merge branch 'cassandra-5.0' into trunk
1f91e99223b0d1b7ed8390400d4a06ac08e4aa85;2009-03-02 07:57:22 +0000;git-svn-id: https://svn.apache.org/repos/asf/incubator/cassandra/trunk@749218 13f79535-47bb-0310-9956-ffa450edef68


# Analisa os issues que foram salvos em commits

In [10]:
with open('/content/cassandra/all_commits_msg.txt', mode='r') as f:
  conteudo = f.read()

list_issue_number = extract_cassandra_issue(conteudo)

clean_list_issue_number = []
for each in list_issue_number:
  item = each.split()[0]
  item = item.replace(")", "")
  if item != 'CASSANDRA-' and item != 'CASSANDRA-CASSANDRA':
    clean_list_issue_number.append(item)

total_items = len(clean_list_issue_number)

In [22]:
i = 0
iterable_list_issue_number = tqdm.tqdm(clean_list_issue_number, total=total_items)
# Faz a limpeza para garantir o id do Issue tracker
list_cassandra_issues_analyzed = []
for each in iterable_list_issue_number:
  issue = each.split()
  if len(issue) == 1:
    elemento = issue[0].replace('"', '')
    elemento = elemento.replace(")", "")
    elemento = elemento.replace(".", "")
    list_cassandra_issues_analyzed.append(elemento)
  percentage = (i + 1) / total_items * 100
  iterable_list_issue_number.set_description(f"Progress Message Analysis")
  i += 1

Progress Message Analysis: 100%|██████████| 4702/4702 [00:08<00:00, 523.82it/s]


In [32]:
print(f'{len(list_cassandra_issues_analyzed)} commits com Id de issue: {list_cassandra_issues_analyzed}')

4702 commits com Id de issue: ['CASSANDRA-codec', 'CASSANDRA-18606', 'CASSANDRA-18745', 'CASSANDRA-18439', 'CASSANDRA-16905', 'CASSANDRA-16905', 'CASSANDRA-14227', 'CASSANDRA-18705', 'CASSANDRA-17687', 'CASSANDRA-fix', 'CASSANDRA-18329', 'CASSANDRA-18690', 'CASSANDRA-18180', 'CASSANDRA-warnings', 'CASSANDRA-17884', 'CASSANDRA-18285', 'CASSANDRA-18511', 'CASSANDRA-fix', 'CASSANDRA-fix', 'CASSANDRA-18436', 'CASSANDRA-18267', 'CASSANDRA-18395', 'CASSANDRA-18438', 'CASSANDRA-17199;', 'CASSANDRA-17199;', 'CASSANDRA-18242', 'CASSANDRA-18247', 'CASSANDRA-18258', 'CASSANDRA-18242', 'CASSANDRA-18252', 'CASSANDRA-18050', 'CASSANDRA-large', 'CASSANDRA-18209', 'CASSANDRA-8210522', 'CASSANDRA-18127', 'CASSANDRA-17976', 'CASSANDRA-18001', 'CASSANDRA-17915', 'CASSANDRA-17950]', 'CASSANDRA-17711:', 'CASSANDRA-fix', 'CASSANDRA-17912', 'CASSANDRA-16413', 'CASSANDRA-17873', 'CASSANDRA-17750', 'CASSANDRA-completing', 'CASSANDRA-17904', 'CASSANDRA-17835', 'CASSANDRA-17778', 'CASSANDRA-17737', 'CASSANDRA-17

# Gera intervalo de datas entre o 1o e o último commit

In [23]:
dates = generate_intervals_between_dates(date1=(2009,3,2), date2=(2023,9,12), distance=120)
convert_interval_dates(dates)

[('2009/03/02', '2009/06/29'),
 ('2009/06/30', '2009/10/27'),
 ('2009/10/28', '2010/02/24'),
 ('2010/02/25', '2010/06/24'),
 ('2010/06/25', '2010/10/22'),
 ('2010/10/23', '2011/02/19'),
 ('2011/02/20', '2011/06/19'),
 ('2011/06/20', '2011/10/17'),
 ('2011/10/18', '2012/02/14'),
 ('2012/02/15', '2012/06/13'),
 ('2012/06/14', '2012/10/11'),
 ('2012/10/12', '2013/02/08'),
 ('2013/02/09', '2013/06/08'),
 ('2013/06/09', '2013/10/06'),
 ('2013/10/07', '2014/02/03'),
 ('2014/02/04', '2014/06/03'),
 ('2014/06/04', '2014/10/01'),
 ('2014/10/02', '2015/01/29'),
 ('2015/01/30', '2015/05/29'),
 ('2015/05/30', '2015/09/26'),
 ('2015/09/27', '2016/01/24'),
 ('2016/01/25', '2016/05/23'),
 ('2016/05/24', '2016/09/20'),
 ('2016/09/21', '2017/01/18'),
 ('2017/01/19', '2017/05/18'),
 ('2017/05/19', '2017/09/15'),
 ('2017/09/16', '2018/01/13'),
 ('2018/01/14', '2018/05/13'),
 ('2018/05/14', '2018/09/10'),
 ('2018/09/11', '2019/01/08'),
 ('2019/01/09', '2019/05/08'),
 ('2019/05/09', '2019/09/05'),
 ('2019/

In [24]:
gera_lista_sentencas(project_name='CASSANDRA', dates=dates)

['project=CASSANDRA and created>="2009/03/02" and created<="2009/06/29"',
 'project=CASSANDRA and created>="2009/06/30" and created<="2009/10/27"',
 'project=CASSANDRA and created>="2009/10/28" and created<="2010/02/24"',
 'project=CASSANDRA and created>="2010/02/25" and created<="2010/06/24"',
 'project=CASSANDRA and created>="2010/06/25" and created<="2010/10/22"',
 'project=CASSANDRA and created>="2010/10/23" and created<="2011/02/19"',
 'project=CASSANDRA and created>="2011/02/20" and created<="2011/06/19"',
 'project=CASSANDRA and created>="2011/06/20" and created<="2011/10/17"',
 'project=CASSANDRA and created>="2011/10/18" and created<="2012/02/14"',
 'project=CASSANDRA and created>="2012/02/15" and created<="2012/06/13"',
 'project=CASSANDRA and created>="2012/06/14" and created<="2012/10/11"',
 'project=CASSANDRA and created>="2012/10/12" and created<="2013/02/08"',
 'project=CASSANDRA and created>="2013/02/09" and created<="2013/06/08"',
 'project=CASSANDRA and created>="2013

# Coleta os issues do Servidor do Jira

In [25]:
issues_by_date_2009_2023 = lista_acesso_jira(lista_sentencas=gera_lista_sentencas(project_name='CASSANDRA', dates=dates))

Aguarde...
project=CASSANDRA and created>="2009/03/02" and created<="2009/06/29" 263
project=CASSANDRA and created>="2009/06/30" and created<="2009/10/27" 251
project=CASSANDRA and created>="2009/10/28" and created<="2010/02/24" 310
project=CASSANDRA and created>="2010/02/25" and created<="2010/06/24" 388
project=CASSANDRA and created>="2010/06/25" and created<="2010/10/22" 419
project=CASSANDRA and created>="2010/10/23" and created<="2011/02/19" 545
project=CASSANDRA and created>="2011/02/20" and created<="2011/06/19" 593
project=CASSANDRA and created>="2011/06/20" and created<="2011/10/17" 573
project=CASSANDRA and created>="2011/10/18" and created<="2012/02/14" 532
project=CASSANDRA and created>="2012/02/15" and created<="2012/06/13" 423
project=CASSANDRA and created>="2012/06/14" and created<="2012/10/11" 441
project=CASSANDRA and created>="2012/10/12" and created<="2013/02/08" 433
project=CASSANDRA and created>="2013/02/09" and created<="2013/06/08" 386
project=CASSANDRA and creat

In [27]:
# Use a list comprehension to concatenate the sublists
concatenated_list = [item for sublist in issues_by_date_2009_2023 for item in sublist]
len(concatenated_list)

18570

# Cria um Dataframe referente aos issues coletados

In [28]:
dict_all_issues = {}

list_key, list_summary, list_status, list_description, list_priority, list_timeestimate, list_timespent, list_issuetype = list(), list(), list(), list(), list(), list(), list(), list()

i = 1
for issue in concatenated_list:
    #print(f'{i}, Key: {issue.key}, Summary: {issue.fields.summary}, Status: {issue.fields.status}, Priority: {issue.fields.priority}, Timespent: {issue.fields.timespent}, Time estimate: {issue.fields.timeestimate}')
    list_key.append(issue.key)
    list_summary.append(issue.fields.summary)
    list_status.append(issue.fields.status)
    list_description.append(issue.fields.description)
    list_priority.append(issue.fields.priority)
    list_timespent.append(issue.fields.timespent)
    list_timeestimate.append(issue.fields.timeestimate)
    list_issuetype.append(issue.fields.issuetype)
    i += 1

dict_all_issues['key'] = list_key
dict_all_issues['issue_type'] = list_issuetype
dict_all_issues['summary'] = list_summary
dict_all_issues['status'] = list_status
dict_all_issues['description'] = list_description
dict_all_issues['priority'] = list_priority
dict_all_issues['timespent'] = list_timespent
dict_all_issues['timeestimate'] = list_timeestimate

In [29]:
df_cassandra_all_issues = pd.DataFrame(dict_all_issues)
df_cassandra_all_issues

,key,issue_type,summary,status,description,priority,timespent,timeestimate
0,CASSANDRA-263,Bug,get_slice needs to support desc from last column,Resolved,At the moment there's no way to ask for a slic...,Normal,NaN,NaN
1,CASSANDRA-262,Bug,get_slice needs to allow returning all columns,Resolved,Right now get_slice requires you to enter a 'l...,Normal,NaN,NaN
2,CASSANDRA-261,Bug,get_slice needs offset + limit,Resolved,Right now get_slice does not allow you to prov...,Normal,NaN,NaN
3,CASSANDRA-260,Bug,Unable to read all columns in a column family ...,Resolved,Since r788516 the typical 'get all columns for...,Normal,NaN,NaN
4,CASSANDRA-259,New Feature,LRU cache for key positions,Resolved,"add cache like the old touch cache, but workin...",Normal,NaN,NaN
...,...,...,...,...,...,...,...,...
18565,CASSANDRA-18769,Bug,Fix flaky test: org.apache.cassandra.cql3.Memt...,Resolved,{noformat}\r\njava.lang.reflect.InaccessibleOb...,Normal,NaN,NaN
18566,CASSANDRA-18768,New Feature,Harry changes required for CEP-21 testing (NTS...,Triage Needed,None,Normal,NaN,NaN
18567,CASSANDRA-18767,Bug,tablestats should show speculative retries,Resolved,NodeProbe makes [provisions|https://github.com...,Normal,NaN,NaN
18568,CASSANDRA-18766,Bug,high speculative retries on v4.1.3,Open,There are up to 10+ times higher speculative r...,Normal,NaN,NaN


In [30]:
lista_key_cassandra_all_issues = df_cassandra_all_issues.key.unique().tolist()
len(lista_key_cassandra_all_issues)

18570

# Gera os dicionários com commits e arquivos modificados

In [33]:
repository_path = "cassandra"
total_commit_count = get_total_commit_count(repository_path)
print("Total number of commits:", total_commit_count)

Total number of commits: 29152


In [34]:
# 4. Extract Commit
import tqdm

total_commits = total_commit_count
dict_commit_modified_files = {}
i = 0

iterable_commits = tqdm.tqdm(pydriller.Repository("cassandra").traverse_commits(), total=total_commits)

for commit in iterable_commits:
  list_modified_files = []
  for m in commit.modified_files:
    list_modified_files.append(m.filename)
  qtd_modified_files = len(list_modified_files)
  commit_message = commit.msg
  elemento = (commit_message, list_modified_files, qtd_modified_files)
  dict_commit_modified_files[commit.hash] = elemento
  percentage = (i + 1) / total_commits * 100
  iterable_commits.set_description(f"Progress Commit Analysis")
  i += 1

Progress Commit Analysis: 100%|██████████| 29152/29152 [07:05<00:00, 68.57it/s]


In [35]:
# TODO: vale colocar uma barra de progresso
dict_message_commits_valido = {}

for k,v in dict_commit_modified_files.items():
  if v[0] is not None:
    for cassandra_issue in list_cassandra_issues_analyzed:
      if cassandra_issue != 'CASSANDRA-' and cassandra_issue in str(v[0]):
        dict_message_commits_valido[k] = cassandra_issue
        break

In [36]:
# TODO: vale colocar uma barra de progresso
dict_key_commit_valido = {}
list_commits_by_key = []
i = 0
for each_issue in lista_key_cassandra_all_issues:
  list_commits_by_key = []
  for k,v in dict_message_commits_valido.items():
    if each_issue == v:
      i +=1
      list_commits_by_key.append(k)
  dict_key_commit_valido[each_issue] = list_commits_by_key

len(dict_key_commit_valido)

18570

In [37]:
key_commits_by_key = list(dict_key_commit_valido.keys() )
commits_commits_by_key = list(dict_key_commit_valido.values() )

dict_key_commit_valido_temp = {
    'key': key_commits_by_key,
    'commits': commits_commits_by_key
}

df_key_commit_valido = pd.DataFrame(data=dict_key_commit_valido_temp)
df_key_commit_valido

,key,commits
0,CASSANDRA-263,[]
1,CASSANDRA-262,[]
2,CASSANDRA-261,[]
3,CASSANDRA-260,[]
4,CASSANDRA-259,"[26aefa2f4a04083c42f5338eda24a5bb09ab04ed, 013..."
...,...,...
18565,CASSANDRA-18769,[]
18566,CASSANDRA-18768,[]
18567,CASSANDRA-18767,[]
18568,CASSANDRA-18766,[]


In [38]:
def get_modified_files_from_commit(key):
  list_files = []
  MESSAGE = 0
  MODIFIED_FILES = 1
  QTD_MODIFIED_FILES = 2
  for each in key:
    if each in dict_commit_modified_files.keys():
      if dict_commit_modified_files[each] is not None:
        list_files.append(dict_commit_modified_files[each][MODIFIED_FILES])
  return list_files

In [39]:
df_key_commit_valido['modified_files'] = df_key_commit_valido['commits'].apply(get_modified_files_from_commit)
df_cassandra_all_issues['commits'] = df_key_commit_valido['commits']
df_cassandra_all_issues['modified_files'] = df_key_commit_valido['modified_files']
df_cassandra_all_issues[['key', 'issue_type', 'commits', 'modified_files']]

,key,issue_type,commits,modified_files
0,CASSANDRA-263,Bug,[],[]
1,CASSANDRA-262,Bug,[],[]
2,CASSANDRA-261,Bug,[],[]
3,CASSANDRA-260,Bug,[],[]
4,CASSANDRA-259,New Feature,"[26aefa2f4a04083c42f5338eda24a5bb09ab04ed, 013...","[[ColumnFamilyStore.java, SSTable.java], [stor..."
...,...,...,...,...
18565,CASSANDRA-18769,Bug,[],[]
18566,CASSANDRA-18768,New Feature,[],[]
18567,CASSANDRA-18767,Bug,[],[]
18568,CASSANDRA-18766,Bug,[],[]


In [40]:
df_cassandra_all_issues

,key,issue_type,summary,status,description,priority,timespent,timeestimate,commits,modified_files
0,CASSANDRA-263,Bug,get_slice needs to support desc from last column,Resolved,At the moment there's no way to ask for a slic...,Normal,NaN,NaN,[],[]
1,CASSANDRA-262,Bug,get_slice needs to allow returning all columns,Resolved,Right now get_slice requires you to enter a 'l...,Normal,NaN,NaN,[],[]
2,CASSANDRA-261,Bug,get_slice needs offset + limit,Resolved,Right now get_slice does not allow you to prov...,Normal,NaN,NaN,[],[]
3,CASSANDRA-260,Bug,Unable to read all columns in a column family ...,Resolved,Since r788516 the typical 'get all columns for...,Normal,NaN,NaN,[],[]
4,CASSANDRA-259,New Feature,LRU cache for key positions,Resolved,"add cache like the old touch cache, but workin...",Normal,NaN,NaN,"[26aefa2f4a04083c42f5338eda24a5bb09ab04ed, 013...","[[ColumnFamilyStore.java, SSTable.java], [stor..."
...,...,...,...,...,...,...,...,...,...,...
18565,CASSANDRA-18769,Bug,Fix flaky test: org.apache.cassandra.cql3.Memt...,Resolved,{noformat}\r\njava.lang.reflect.InaccessibleOb...,Normal,NaN,NaN,[],[]
18566,CASSANDRA-18768,New Feature,Harry changes required for CEP-21 testing (NTS...,Triage Needed,None,Normal,NaN,NaN,[],[]
18567,CASSANDRA-18767,Bug,tablestats should show speculative retries,Resolved,NodeProbe makes [provisions|https://github.com...,Normal,NaN,NaN,[],[]
18568,CASSANDRA-18766,Bug,high speculative retries on v4.1.3,Open,There are up to 10+ times higher speculative r...,Normal,NaN,NaN,[],[]


# Calcula a frequência dos arquivos nos issues

In [41]:
# @title
list_temp_key, list_temp_commits, list_temp_modified_files = list(), list(), list()

list_temp_commit_key = list()

for i in df_cassandra_all_issues.index:
  consulta_commits = df_cassandra_all_issues['commits'][i]
  if len(consulta_commits) > 0:
    for each_commit in consulta_commits:
      #print(df_cassandra_all_issues['key'][i], df_cassandra_all_issues['commits'][i], df_cassandra_all_issues['modified_files'][i])
      if df_cassandra_all_issues['key'][i] not in list_temp_key:
        list_temp_key.append(df_cassandra_all_issues['key'][i])
        commits_separated_by_commas = ",".join(df_cassandra_all_issues['commits'][i])
        list_temp_commits.append(commits_separated_by_commas)
        list_temp_modified_files.append(df_cassandra_all_issues['modified_files'][i])
        l_temp_commits = commits_separated_by_commas.split(',')
        for e in l_temp_commits:
          elemento = (e, df_cassandra_all_issues['key'][i])
          list_temp_commit_key.append(elemento)

dict_temp_2 = {
    'keys':list_temp_key, 'commits':list_temp_commits, 'modified_files':list_temp_modified_files
}

df_temp_2 = pd.DataFrame(dict_temp_2)
df_temp_2

,keys,commits,modified_files
0,CASSANDRA-259,"26aefa2f4a04083c42f5338eda24a5bb09ab04ed,013d5...","[[ColumnFamilyStore.java, SSTable.java], [stor..."
1,CASSANDRA-258,af82ca7ae16751641c051ef0ed7f76d01024d76e,"[[cassandra.thrift, Cassandra.java, CliClient...."
2,CASSANDRA-256,"717b111f7bbd0ebe047ba3a1c718302d622121ec,da003...","[[storage-conf.xml], [build.xml]]"
3,CASSANDRA-255,"27d66d8269d5f52dd29b98cac0b41574291f16ad,963a1...","[[Column.java, ColumnFamily.java, SuperColumn...."
4,CASSANDRA-254,"4283f0d879ed551859650f94d6eb0009daaa5361,4e7f8...","[[SSTable.java], [BinaryMemtable.java, ColumnF..."
...,...,...,...
2674,CASSANDRA-18745,"6bb585bf5db106322c9569d76f3178c79ed53d5c,65ee0...","[[build-cqlsh.xml, CHANGES.txt, cqlsh.py, buil..."
2675,CASSANDRA-18705,"89fe2223b4e1118dbd11cfd00f2a7b7d12940502,aa9b5...","[[run-tests.sh, config.yml, config.yml.FREE, c..."
2676,CASSANDRA-18690,"c1bf002234bd44bce120658e9b08b3d67901e5bb,738df...","[[build.xml], [build.xml]]"
2677,CASSANDRA-18606,"1e2c88fff832d891b296165e9adda786182e850d,bf391...","[[Makefile, index.adoc, java_verify.result, na..."


In [51]:
# 4. Extract Commit and Issue Data
#commits = list(RepositoryMining(github_repo_url).traverse_commits())

# Assuming you have a Jira API or library to fetch issue data
# issues = fetch_jira_issues(jira_url)

In [42]:
# 5. Data preparation
# Filter commits made for this repository
# repository_commits = [commit for commit in commits if commit.project_name == 'your_repository']
repository_commits = dict_commit_modified_files.keys()

In [43]:
# Extract modified files from each commit
#modified_files = [(commit.hash, file) for commit in repository_commits for file in commit.modifications]
modified_files = []
for k, v in dict_commit_modified_files.items():
  for each in v[1]:
    elemento = (k, each)
    modified_files.append(elemento)

In [44]:
# Assuming you have a function to extract the issues mentioned in commits
#commit_issues = extract_issues_mentioned_in_commits(repository_commits, issues)
commit_issues = list_temp_commit_key

In [45]:
# Create a Pandas DataFrame for further analysis
commit_df = pd.DataFrame(modified_files, columns=['CommitHash', 'ModifiedFile'])
issue_df = pd.DataFrame(commit_issues, columns=['CommitHash', 'IssueID'])

In [46]:
commit_df

,CommitHash,ModifiedFile
0,1f91e99223b0d1b7ed8390400d4a06ac08e4aa85,JSAP-2.1.jar
1,1f91e99223b0d1b7ed8390400d4a06ac08e4aa85,activation.jar
2,1f91e99223b0d1b7ed8390400d4a06ac08e4aa85,antlr-2.7.6.jar
3,1f91e99223b0d1b7ed8390400d4a06ac08e4aa85,antlr-2.7.7.jar
4,1f91e99223b0d1b7ed8390400d4a06ac08e4aa85,antlr-3.0.1.jar
...,...,...
98812,6708499e4c930f7733894e6076b925fb151d9251,configuring.adoc
98813,6708499e4c930f7733894e6076b925fb151d9251,cass_env_sh_file.adoc
98814,6708499e4c930f7733894e6076b925fb151d9251,metrics.adoc
98815,6708499e4c930f7733894e6076b925fb151d9251,CassandraRelevantProperties.java


In [47]:
issue_df

,CommitHash,IssueID
0,26aefa2f4a04083c42f5338eda24a5bb09ab04ed,CASSANDRA-259
1,013d59fc00d7d241abdf5bc0448ce55425c5f112,CASSANDRA-259
2,d6c0b0f2596de169186d3584474d8d5a34c3c9de,CASSANDRA-259
3,b54de30a0226daafaa6ecf0257371e6e8c2a0742,CASSANDRA-259
4,af82ca7ae16751641c051ef0ed7f76d01024d76e,CASSANDRA-258
...,...,...
10359,738dfde73560d289218f3afa4103a37e9f401757,CASSANDRA-18690
10360,1e2c88fff832d891b296165e9adda786182e850d,CASSANDRA-18606
10361,bf39188cdc6f8c71d247f9519ef126d2db7dd913,CASSANDRA-18606
10362,43ec1843918aba9e81d3c2dc1433a1ef4740a51f,CASSANDRA-18511


In [48]:
# 6. Correlation Analysis
# Group by files and count how many times each file appears in issues
file_issue_counts = issue_df['IssueID'].groupby(issue_df['CommitHash']).count().reset_index()

In [49]:
# Merge with commit data
merged_data = pd.merge(commit_df, file_issue_counts, on='CommitHash', how='left')

In [50]:
# Create a table showing files and the number of times they appeared in issues
file_issue_summary = merged_data.groupby('ModifiedFile')['IssueID'].sum().reset_index()
file_issue_summary.rename(columns={'IssueID': 'TimesAppearedInIssues'}, inplace=True)

In [55]:
file_issue_summary.sort_values(by='TimesAppearedInIssues', ascending=False).head(50)

,ModifiedFile,TimesAppearedInIssues
683,CHANGES.txt,5769.0
5476,StorageService.java,953.0
1076,ColumnFamilyStore.java,820.0
1603,DatabaseDescriptor.java,652.0
6459,build.xml,589.0
3649,NEWS.txt,482.0
6633,cassandra.yaml,471.0
5473,StorageProxy.java,454.0
838,CassandraServer.java,403.0
1251,CompactionManager.java,403.0


In [ ]:
# 7. Analyze Results
# DataFrame (file_issue_summary) that contains information about how many times each modified file appeared in issues.
# Analyze and visualize this data to draw conclusions about the correlation between file modifications and issues.